<a href="https://colab.research.google.com/github/Aryan-401/GoogleColabProjects/blob/main/CIFAR_10_Convolutional_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
num_epochs = 4
batch_size = 4
learning_rate = 0.001

transform = transforms.Compose(
    [
        transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)


In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download =True, transform = transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download =True, transform = transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    self.conf1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conf2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  def forward(self, x):
    x = self.pool(F.relu(self.conf1(x)))
    x = self.pool(F.relu(self.conf2(x)))
    x = x.view(-1, 16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x


In [ ]:
model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
model.to(device)
for epoch in range(num_epochs):
  for i,(images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    loss = criterion(outputs, labels)
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    if (i+1)%2000 == 0:
      print(f'epoch #{epoch+1}/{num_epochs} | Step:{(i+1)}/{n_total_steps} | Loss:{loss.item():.4f}')
print("Finished Training")

epoch #1/4 | Step:2000/10000 | Loss:2.3082
epoch #1/4 | Step:4000/10000 | Loss:2.3053
epoch #1/4 | Step:6000/10000 | Loss:2.2728
epoch #1/4 | Step:8000/10000 | Loss:2.2948
epoch #1/4 | Step:10000/10000 | Loss:2.2597
epoch #2/4 | Step:2000/10000 | Loss:2.1894
epoch #2/4 | Step:4000/10000 | Loss:1.8801
epoch #2/4 | Step:6000/10000 | Loss:2.0926
epoch #2/4 | Step:8000/10000 | Loss:2.1120
epoch #2/4 | Step:10000/10000 | Loss:1.3645
epoch #3/4 | Step:2000/10000 | Loss:1.9317
epoch #3/4 | Step:4000/10000 | Loss:1.9912
epoch #3/4 | Step:6000/10000 | Loss:1.3274
epoch #3/4 | Step:8000/10000 | Loss:0.8921
epoch #3/4 | Step:10000/10000 | Loss:1.3863
epoch #4/4 | Step:2000/10000 | Loss:1.7328
epoch #4/4 | Step:4000/10000 | Loss:1.7411
epoch #4/4 | Step:6000/10000 | Loss:1.2210
epoch #4/4 | Step:8000/10000 | Loss:1.7922
epoch #4/4 | Step:10000/10000 | Loss:0.9294
Finished Training


In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 41.97 %
Accuracy of plane: 59.42211055276382 %
Accuracy of car: 69.36708860759494 %
Accuracy of bird: 8.663366336633663 %
Accuracy of cat: 37.37623762376238 %
Accuracy of deer: 25.985221674876847 %
Accuracy of dog: 35.57814485387548 %
Accuracy of frog: 50.92936802973978 %
Accuracy of horse: 47.3551637279597 %
Accuracy of ship: 39.75 %
Accuracy of truck: 44.987468671679196 %
